# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-aa886f28f0-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Tv series explorative search") 

## Workflow 4


Consider the following exploratory scenario:


> we are interested in the TV series "How I met your mother" and we want to investigate the main aspects related to the actors and directors involved in the production, know the numerber of seasons and check what are the episodes which got the higher success/impact.


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P4969`    | derivative work      | predicate |
| `wd:Q147235` | How I met your mother        | node |
| `wd:Q23831` | The Office (US)        | node |



Also consider

```
wd:Q23831 ?p ?obj .
```

is the BGP to retrieve all **properties of The Office (US)**

The workload should


1. Return the number of seasons and episodes per season of the tv series

2. Get the number of episodes in which the cast members played a role. Who are the most present actors?

3. Check who is the actor who acted in more films while working on "How I met your mother" and who is the actor who participated in more films after the end of the tv series.

4. Compare HIMYM with the tv series "The Office (US)" in terms of number of seasons, episods and cast members.

5. Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#:~:text=Kevin%20Bacon%20himself%20has%20a,Bacon%20number%20is%20N%2B1.) equal to 2

In [2]:
# start your workflow here

In [3]:
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q23831 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P449'), ('pname', 'original broadcaster'), ('o', 'http://www.wikidata.org/entity/Q13974'), ('oname', 'NBC')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('pname', 'original language of film or TV show'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('oname', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pname', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q21664088'), ('oname', 'two-part episode')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pname', 'country of origin'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oname', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q5398426'), ('oname', 'television series')]
[('p', 'http://www.wikidata.org/prop/direct/P437'), ('pname', 'distribution format'), ('o', 'http://www.wikidata.org/entity/Q723685'), ('oname', 'video on demand')]
[('

72

In [4]:

queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q23831 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?oname, "episode", "i") 
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1811'), ('pname', 'list of episodes'), ('o', 'http://www.wikidata.org/entity/Q1361122'), ('oname', 'list of The Office (U.S. TV series) episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pname', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q21664088'), ('oname', 'two-part episode')]


2

# Task 1
Return the number of seasons and episodes per season of the tv series

In [5]:

queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q23831 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?oname, "season", "i") 
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q3468906'), ('oname', 'The Office, season 4')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q3730261'), ('oname', 'The Office, season 6')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q3468601'), ('oname', 'The Office, season 2')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q3730253'), ('oname', 'The Office, season 9')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q3465812'), ('oname', 'The Office, season 1')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q3730255'), ('oname', 'The Office, season 8')]
[('p', 'http://www.wikidata.org/

9

In [6]:
#P527 seasons

queryString = """
SELECT DISTINCT ?season ?seasonname
WHERE { 

    wd:Q23831 wdt:P527 ?season .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?season <http://schema.org/name> ?seasonname .
    FILTER regex(?seasonname, "season", "i") 
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('season', 'http://www.wikidata.org/entity/Q3468906'), ('seasonname', 'The Office, season 4')]
[('season', 'http://www.wikidata.org/entity/Q3730261'), ('seasonname', 'The Office, season 6')]
[('season', 'http://www.wikidata.org/entity/Q3468601'), ('seasonname', 'The Office, season 2')]
[('season', 'http://www.wikidata.org/entity/Q3730253'), ('seasonname', 'The Office, season 9')]
[('season', 'http://www.wikidata.org/entity/Q3465812'), ('seasonname', 'The Office, season 1')]
[('season', 'http://www.wikidata.org/entity/Q3730255'), ('seasonname', 'The Office, season 8')]
[('season', 'http://www.wikidata.org/entity/Q3468797'), ('seasonname', 'The Office, season 3')]
[('season', 'http://www.wikidata.org/entity/Q3730263'), ('seasonname', 'The Office, season 7')]
[('season', 'http://www.wikidata.org/entity/Q3468990'), ('seasonname', 'The Office, season 5')]


9

In [7]:
#Q3468906 season 4
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q3468906 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('pname', 'original language of film or TV show'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('oname', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pname', 'country of origin'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oname', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('pname', 'part of the series'), ('o', 'http://www.wikidata.org/entity/Q23831'), ('oname', 'The Office')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q3464665'), ('oname', 'television series season')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q16744428'), ('oname', 'Job Fair')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q50379716'), ('oname', 'Goodbye, Toby (part 1)')]
[('p', 'http://w

23

In [8]:
#Q3468906 season 4
queryString = """
SELECT DISTINCT ?episode ?episodename
WHERE { 

    wd:Q3468906 wdt:P527 ?episode .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?episode <http://schema.org/name> ?episodename .

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('episode', 'http://www.wikidata.org/entity/Q7033383'), ('episodename', 'Night Out')]
[('episode', 'http://www.wikidata.org/entity/Q50379638'), ('episodename', 'Dunder Mifflin Infinity (part 1)')]
[('episode', 'http://www.wikidata.org/entity/Q50379682'), ('episodename', 'Money (part 1)')]
[('episode', 'http://www.wikidata.org/entity/Q50379663'), ('episodename', 'Launch Party (part 1)')]
[('episode', 'http://www.wikidata.org/entity/Q50379716'), ('episodename', 'Goodbye, Toby (part 1)')]
[('episode', 'http://www.wikidata.org/entity/Q5278408'), ('episodename', 'Dinner Party')]
[('episode', 'http://www.wikidata.org/entity/Q6663943'), ('episodename', 'Local Ad')]
[('episode', 'http://www.wikidata.org/entity/Q7647298'), ('episodename', 'Survivor Man')]
[('episode', 'http://www.wikidata.org/entity/Q50379664'), ('episodename', 'Launch Party (part 2)')]
[('episode', 'http://www.wikidata.org/entity/Q16744428'), ('episodename', 'Job Fair')]
[('episode', 'http://www.wikidata.org/entity/Q5

19

In [9]:
#Q7729754, ('oname', 'The Deposition')] season 4 episode ?
queryString = """
SELECT *
WHERE { 

    wd:Q7729754 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('pname', 'original language of film or TV show'), ('oname', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('o', 'http://www.wikidata.org/entity/Q21191270'), ('pname', 'instance of'), ('oname', 'television series episode')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('o', 'http://www.wikidata.org/entity/Q30'), ('pname', 'country of origin'), ('oname', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P437'), ('o', 'http://www.wikidata.org/entity/Q723685'), ('pname', 'distribution format'), ('oname', 'video on demand')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('o', 'http://www.wikidata.org/entity/Q23831'), ('pname', 'part of the series'), ('oname', 'The Office')]
[('p', 'http://www.wikidata.org/prop/direct/P4908'), ('o', 'http://www.wikidata.org/entity/Q3468906'), ('pname', 'season'), ('oname', 'The Office, season 4')]
[(

11

In [10]:
#Q7729754, ('oname', 'The Deposition')] season 4 episode ?
queryString = """
SELECT *
WHERE { 

    ?p ?o wd:Q7729754 .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q5278408'), ('o', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'Dinner Party'), ('oname', 'follows')]
[('p', 'http://www.wikidata.org/entity/Q7647298'), ('o', 'http://www.wikidata.org/prop/direct/P156'), ('pname', 'Survivor Man'), ('oname', 'followed by')]
[('p', 'http://www.wikidata.org/entity/Q3468906'), ('o', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'The Office, season 4'), ('oname', 'has part')]


3

In [11]:
#Count seasons of The Office

queryString = """
SELECT (COUNT (DISTINCT ?season) AS ?countseason) 
WHERE { 

    wd:Q23831 wdt:P527 ?season .
    # Labels
    ?season <http://schema.org/name> ?seasonname .
    FILTER regex(?seasonname, "season", "i") 
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('countseason', '9')]


1

In [12]:
#Count episodes of a season of The Office

queryString = """
SELECT (COUNT (DISTINCT ?episode) AS ?countepisode) 
WHERE { 
    wd:Q23831 wdt:P527 ?season .
    ?season wdt:P527 ?episode .
    # Labels
    ?season <http://schema.org/name> ?seasonname .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('countepisode', '201')]


1

## Return the number of seasons and episodes per season of the tv series

In [13]:
#Count episodes of a season of The Office and show the number of seasons

queryString = """
    SELECT ?seasonname ?countepisode WHERE {
      {
        SELECT ?season (COUNT (DISTINCT ?episode) AS ?countepisode) 
        WHERE { 
            wd:Q23831 wdt:P527 ?season .
            ?season wdt:P527 ?episode .

                } 
    GROUP BY ?season
    } .
     ## labels   
    ?season <http://schema.org/name> ?seasonname .
    }
    ORDER BY (?seasonname) ?countepisode


"""

print("Results")
run_query(queryString)

Results
[('seasonname', 'The Office, season 1'), ('countepisode', '6')]
[('seasonname', 'The Office, season 2'), ('countepisode', '22')]
[('seasonname', 'The Office, season 3'), ('countepisode', '25')]
[('seasonname', 'The Office, season 4'), ('countepisode', '19')]
[('seasonname', 'The Office, season 5'), ('countepisode', '28')]
[('seasonname', 'The Office, season 6'), ('countepisode', '26')]
[('seasonname', 'The Office, season 7'), ('countepisode', '26')]
[('seasonname', 'The Office, season 8'), ('countepisode', '24')]
[('seasonname', 'The Office, season 9'), ('countepisode', '25')]


9

In [14]:
#Count episodes of a season of HIMYM and show the number of seasons

queryString = """
    SELECT ?countepisode ?seasonname WHERE {
      {
        SELECT ?season (COUNT (DISTINCT ?episode) AS ?countepisode) 
        WHERE { 
            wd:Q147235 wdt:P527 ?season .
            ?season wdt:P527 ?episode .

                } 
    GROUP BY ?season
    } .
     ## labels   
    ?season <http://schema.org/name> ?seasonname .
    }
    ORDER BY (?countepisode) ?seasonname 


"""

print("Results")
run_query(queryString)

Results
[('countepisode', '20'), ('seasonname', 'How I Met Your Mother, season 3')]
[('countepisode', '22'), ('seasonname', 'How I Met Your Mother, season 1')]
[('countepisode', '22'), ('seasonname', 'How I Met Your Mother, season 2')]
[('countepisode', '24'), ('seasonname', 'How I Met Your Mother, season 9')]
[('countepisode', '24'), ('seasonname', 'How I Met Your Mother, season 7')]
[('countepisode', '24'), ('seasonname', 'How I Met Your Mother, season 6')]
[('countepisode', '24'), ('seasonname', 'How I Met Your Mother, season 8')]
[('countepisode', '24'), ('seasonname', 'How I Met Your Mother, season 4')]
[('countepisode', '24'), ('seasonname', 'How I Met Your Mother, season 5')]


9

# Task 2

### Show the name of all the characters of both series, their actors, count the number of the characters and the other occupution every actor has.
### filter the acters who are singers too. 

In [15]:
# list of all characters in HIMYM
# wd:Q147235

queryString = """
SELECT *
WHERE { 

    ?p ?o wd:Q147235 .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    # FILTER regex(?pname, "name", "i") 

} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q5398810'), ('o', 'http://www.wikidata.org/prop/direct/P1423'), ('pname', 'Template:How I Met Your Mother'), ('oname', 'template has topic')]
[('p', 'http://www.wikidata.org/entity/Q2332356'), ('o', 'http://www.wikidata.org/prop/direct/P138'), ('pname', 'How I Wet Your Mother'), ('oname', 'named after')]
[('p', 'http://www.wikidata.org/entity/Q20385035'), ('o', 'http://www.wikidata.org/prop/direct/P144'), ('pname', 'Ahla Ayam'), ('oname', 'based on')]
[('p', 'http://www.wikidata.org/entity/Q4208862'), ('o', 'http://www.wikidata.org/prop/direct/P144'), ('pname', 'How I Met Your Mother'), ('oname', 'based on')]
[('p', 'http://www.wikidata.org/entity/Q2347474'), ('o', 'http://www.wikidata.org/prop/direct/P1441'), ('pname', 'Robin Scherbatsky'), ('oname', 'present in work')]
[('p', 'http://www.wikidata.org/entity/Q50383228'), ('o', 'http://www.wikidata.org/prop/direct/P1441'), ('pname', 'Don Frank'), ('oname', 'present in work')]
[('p', 'http:

10

In [16]:
# list of all characters in HIMYM
# wd:Q147235

queryString = """
SELECT ?charactername
WHERE { 

    ?character ?present wd:Q147235 .
    ?character wdt:P1441 wd:Q147235 .
    # Labels
    ?character <http://schema.org/name> ?charactername .
    ?present <http://schema.org/name> ?presentname .

} 
LIMIT 30
"""

print("Results")
run_query(queryString)






### order by number of their episodes

Results
[('charactername', 'Robin Scherbatsky')]
[('charactername', 'Don Frank')]
[('charactername', 'Kevin Venkataraghavan')]
[('charactername', 'Nick Podarutti')]
[('charactername', 'Metro News One')]
[('charactername', 'World Wide News')]
[('charactername', 'Genevieve Scherbatsky')]
[('charactername', 'Katie Scherbatsky')]
[('charactername', 'Max')]
[('charactername', 'Scooby Scooberman')]
[('charactername', 'Gael')]
[('charactername', 'Ted Mosby')]
[('charactername', 'Barney Stinson')]
[('charactername', 'Doctor John Stangel')]
[('charactername', 'Loretta Stinson')]
[('charactername', 'Carl MacLaren')]
[('charactername', 'The Mother')]
[('charactername', "Ted's son")]
[('charactername', "Ted's daughter")]
[('charactername', 'Arthur Hobbs')]
[('charactername', 'Daphne')]
[('charactername', 'James Stinson')]
[('charactername', 'Linus')]
[('charactername', 'Victoria')]
[('charactername', 'Lesbian Robin')]
[('charactername', 'Mexican Wrestler Ted')]
[('charactername', 'Goliath National

30

In [17]:
# Count all characters in HIMYM
# wd:Q147235

queryString = """
 SELECT (COUNT (DISTINCT ?character) AS ?countcharacter) 
    WHERE { 

    ?character ?present wd:Q147235 .
    ?character wdt:P1441 wd:Q147235 .
    # Labels
    ?character <http://schema.org/name> ?charactername .
    ?present <http://schema.org/name> ?presentname .

    } 
    LIMIT 30
"""

print("Results")
run_query(queryString)




Results
[('countcharacter', '114')]


1

In [18]:
# list of all characters in HIMYM
# wd:Q147235

queryString = """
SELECT ?character ?charactername
WHERE { 

    ?character wdt:P1441 wd:Q147235 .
    # Labels
    ?character <http://schema.org/name> ?charactername .


} 
LIMIT 30
"""

print("Results")
run_query(queryString)



Results
[('character', 'http://www.wikidata.org/entity/Q2347474'), ('charactername', 'Robin Scherbatsky')]
[('character', 'http://www.wikidata.org/entity/Q50383228'), ('charactername', 'Don Frank')]
[('character', 'http://www.wikidata.org/entity/Q50383236'), ('charactername', 'Kevin Venkataraghavan')]
[('character', 'http://www.wikidata.org/entity/Q50383247'), ('charactername', 'Nick Podarutti')]
[('character', 'http://www.wikidata.org/entity/Q50384074'), ('charactername', 'Metro News One')]
[('character', 'http://www.wikidata.org/entity/Q50384098'), ('charactername', 'World Wide News')]
[('character', 'http://www.wikidata.org/entity/Q50385677'), ('charactername', 'Genevieve Scherbatsky')]
[('character', 'http://www.wikidata.org/entity/Q50385690'), ('charactername', 'Katie Scherbatsky')]
[('character', 'http://www.wikidata.org/entity/Q50385704'), ('charactername', 'Max')]
[('character', 'http://www.wikidata.org/entity/Q50385707'), ('charactername', 'Scooby Scooberman')]
[('character', 

30

In [19]:
# find the relationship between actor and character
#[('character', 'http://www.wikidata.org/entity/Q2347474'),
# ('charactername', 'Robin Scherbatsky')]

queryString = """
SELECT *
WHERE { 

    ?p ?o wd:Q2347474 .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    # FILTER regex(?pname, "name", "i") 

} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q838512'), ('o', 'http://www.wikidata.org/prop/direct/P26'), ('pname', 'Barney Stinson'), ('oname', 'spouse')]
[('p', 'http://www.wikidata.org/entity/Q50385690'), ('o', 'http://www.wikidata.org/prop/direct/P3373'), ('pname', 'Katie Scherbatsky'), ('oname', 'sibling')]
[('p', 'http://www.wikidata.org/entity/Q50385677'), ('o', 'http://www.wikidata.org/prop/direct/P40'), ('pname', 'Genevieve Scherbatsky'), ('oname', 'child')]
[('p', 'http://www.wikidata.org/entity/Q50383251'), ('o', 'http://www.wikidata.org/prop/direct/P40'), ('pname', 'Robin Scherbatsky Sr.'), ('oname', 'child')]
[('p', 'http://www.wikidata.org/entity/Q50385707'), ('o', 'http://www.wikidata.org/prop/direct/P451'), ('pname', 'Scooby Scooberman'), ('oname', 'unmarried partner')]
[('p', 'http://www.wikidata.org/entity/Q50383247'), ('o', 'http://www.wikidata.org/prop/direct/P451'), ('pname', 'Nick Podarutti'), ('oname', 'unmarried partner')]
[('p', 'http://www.wikidata.org/entit

10

In [20]:
# find the relationship between actor and character
#[('character', 'http://www.wikidata.org/entity/Q2347474'),
# ('charactername', 'Robin Scherbatsky')]

queryString = """
SELECT *
WHERE { 

    wd:Q2347474 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    # FILTER regex(?pname, "name", "i") 

} 
LIMIT 10
"""

print("Results")
run_query(queryString)
# performer : P175

Results
[('p', 'http://www.wikidata.org/prop/direct/P175'), ('o', 'http://www.wikidata.org/entity/Q200566'), ('pname', 'performer'), ('oname', 'Cobie Smulders')]
[('p', 'http://www.wikidata.org/prop/direct/P1441'), ('o', 'http://www.wikidata.org/entity/Q147235'), ('pname', 'present in work'), ('oname', 'How I Met Your Mother')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('o', 'http://www.wikidata.org/entity/Q15632617'), ('pname', 'instance of'), ('oname', 'fictional human')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('o', 'http://www.wikidata.org/entity/Q15773317'), ('pname', 'instance of'), ('oname', 'television character')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('o', 'http://www.wikidata.org/entity/Q16'), ('pname', 'country of citizenship'), ('oname', 'Canada')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('o', 'http://www.wikidata.org/entity/Q177220'), ('pname', 'occupation'), ('oname', 'singer')]
[('p', 'http://www.wikidata.org/prop/direct/P141

10

In [21]:
# find the relationship between actor and character
#[('character', 'http://www.wikidata.org/entity/Q2347474'),
# ('charactername', 'Robin Scherbatsky')]

queryString = """
SELECT ?actor ?actorname
WHERE { 

    wd:Q2347474 wdt:P175 ?actor .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
} 
LIMIT 10
"""

print("Results")
run_query(queryString)
# performer : P175

Results
[('actor', 'http://www.wikidata.org/entity/Q200566'), ('actorname', 'Cobie Smulders')]


1

In [22]:
# actor name and finding its occupation
#[('actor', 'http://www.wikidata.org/entity/Q200566'),
#('actorname', 'Cobie Smulders')]
queryString = """
SELECT *
WHERE { 

    wd:Q200566 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)
# occupation : P106

Results
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('o', 'http://www.wikidata.org/entity/Q15208593'), ('pname', 'given name'), ('oname', 'Francisca')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('o', 'http://www.wikidata.org/entity/Q6119607'), ('pname', 'given name'), ('oname', 'Jacoba')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('o', 'http://www.wikidata.org/entity/Q10798782'), ('pname', 'occupation'), ('oname', 'television actor')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('o', 'http://www.wikidata.org/entity/Q10800557'), ('pname', 'occupation'), ('oname', 'film actor')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('o', 'http://www.wikidata.org/entity/Q16'), ('pname', 'country of citizenship'), ('oname', 'Canada')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('pname', 'languages spoken, written or signed'), ('oname', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('

15

In [23]:
# actor name and finding its occupation
#[('actor', 'http://www.wikidata.org/entity/Q200566'),
#('actorname', 'Cobie Smulders')]
queryString = """
SELECT ?occupationname
WHERE { 

    wd:Q200566 wdt:P106 ?occupation .
    # Labels
    ?occupation <http://schema.org/name> ?occupationname .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)
# occupation : P106

Results
[('occupationname', 'television actor')]
[('occupationname', 'film actor')]
[('occupationname', 'stage actor')]
[('occupationname', 'model')]


4

### HOW I MET YOUR MOTHER

In [24]:
# Show the name of all the characters of both series, their actors,
# 

queryString = """
SELECT ?charactername ?actorname
WHERE { 

    ?character wdt:P1441 wd:Q147235 .
    ?character wdt:P175 ?actor .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?character <http://schema.org/name> ?charactername .
    
} 
LIMIT 30
"""

print("Results")
run_query(queryString)


Results
[('charactername', 'Robin Scherbatsky'), ('actorname', 'Cobie Smulders')]
[('charactername', 'Don Frank'), ('actorname', 'Benjamin Koldyke')]
[('charactername', 'Kevin Venkataraghavan'), ('actorname', 'Kal Penn')]
[('charactername', 'Nick Podarutti'), ('actorname', 'Michael Trucco')]
[('charactername', 'Genevieve Scherbatsky'), ('actorname', 'Tracey Ullman')]
[('charactername', 'Katie Scherbatsky'), ('actorname', 'Lucy Hale')]
[('charactername', 'Katie Scherbatsky'), ('actorname', 'Pamela Darling')]
[('charactername', 'Max'), ('actorname', 'Geoff Stults')]
[('charactername', 'Scooby Scooberman'), ('actorname', 'Robbie Amell')]
[('charactername', 'Gael'), ('actorname', 'Enrique Iglesias')]
[('charactername', 'Ted Mosby'), ('actorname', 'Josh Radnor')]
[('charactername', 'Barney Stinson'), ('actorname', 'Neil Patrick Harris')]
[('charactername', 'Doctor John Stangel'), ('actorname', 'Neil Patrick Harris')]
[('charactername', 'Loretta Stinson'), ('actorname', 'Frances Conroy')]
[(

30

In [25]:
# Show the name of all the characters of both series, their actors,
# count the number of the characters 

queryString = """
SELECT (COUNT (DISTINCT ?character) AS ?countcharacter) 
WHERE { 

    ?character wdt:P1441 wd:Q147235 .
    ?character wdt:P175 ?actor .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?character <http://schema.org/name> ?charactername .
    
} 

"""

print("Results")
run_query(queryString)


Results
[('countcharacter', '109')]


1

In [26]:
# Show the name of all the characters of both series, their actors
# and the other occupution every actor has.

queryString = """
SELECT ?charactername ?actorname ?occupationname
WHERE { 

    ?character wdt:P1441 wd:Q147235 .
    ?character wdt:P175 ?actor .
    ?actor wdt:P106 ?occupation .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?character <http://schema.org/name> ?charactername .
    ?occupation <http://schema.org/name> ?occupationname .

} 
ORDER BY ?charactername
LIMIT 30
"""

print("Results")
run_query(queryString)



Results
[('charactername', "'Crazy' Meg"), ('actorname', 'April Bowlby'), ('occupationname', 'television actor')]
[('charactername', "'Crazy' Meg"), ('actorname', 'April Bowlby'), ('occupationname', 'film actor')]
[('charactername', "'Crazy' Meg"), ('actorname', 'April Bowlby'), ('occupationname', 'actor')]
[('charactername', 'Abby'), ('actorname', 'Britney Spears'), ('occupationname', 'television actor')]
[('charactername', 'Abby'), ('actorname', 'Britney Spears'), ('occupationname', 'film actor')]
[('charactername', 'Abby'), ('actorname', 'Britney Spears'), ('occupationname', 'manufacturer')]
[('charactername', 'Abby'), ('actorname', 'Britney Spears'), ('occupationname', 'singer')]
[('charactername', 'Abby'), ('actorname', 'Britney Spears'), ('occupationname', 'record producer')]
[('charactername', 'Abby'), ('actorname', 'Britney Spears'), ('occupationname', 'stage actor')]
[('charactername', 'Abby'), ('actorname', 'Britney Spears'), ('occupationname', 'music video director')]
[('cha

30

## Singers

In [27]:
# Show the name of all the characters of both series, their actors
# and the other occupution every actor has.

queryString = """
SELECT ?charactername ?actorname ?occupationname
WHERE { 

    ?character wdt:P1441 wd:Q147235 .
    ?character wdt:P175 ?actor .
    ?actor wdt:P106 ?occupation .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?character <http://schema.org/name> ?charactername .
    ?occupation <http://schema.org/name> ?occupationname .
    FILTER regex(?occupationname, "singer", "i") 
} 
ORDER BY ?charactername
LIMIT 30
"""

print("Results")
run_query(queryString)



Results
[('charactername', 'Abby'), ('actorname', 'Britney Spears'), ('occupationname', 'singer')]
[('charactername', 'Abby'), ('actorname', 'Britney Spears'), ('occupationname', 'singer-songwriter')]
[('charactername', 'Abby'), ('actorname', 'Britney Spears'), ('occupationname', 'child singer')]
[('charactername', 'Amy'), ('actorname', 'Mandy Moore'), ('occupationname', 'singer')]
[('charactername', 'Amy'), ('actorname', 'Mandy Moore'), ('occupationname', 'singer-songwriter')]
[('charactername', 'Anita Appleby'), ('actorname', 'Jennifer Lopez'), ('occupationname', 'singer-songwriter')]
[('charactername', 'Barney Stinson'), ('actorname', 'Neil Patrick Harris'), ('occupationname', 'singer')]
[('charactername', 'Becky'), ('actorname', 'Laura Bell Bundy'), ('occupationname', 'singer')]
[('charactername', 'Cathy'), ('actorname', 'Lindsay Price'), ('occupationname', 'singer')]
[('charactername', 'Coat-Check Girl'), ('actorname', 'Jayma Mays'), ('occupationname', 'singer')]
[('charactername'

30

### The office 

In [28]:
# Show the name of all the characters of both series, their actors,
# 

queryString = """
SELECT ?charactername ?actorname
WHERE { 

    ?character wdt:P1441 wd:Q23831 .
    ?character wdt:P175 ?actor .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?character <http://schema.org/name> ?charactername .
    


} 
LIMIT 30
"""

print("Results")
run_query(queryString)


Results
[('charactername', 'Ryan Howard'), ('actorname', 'B. J. Novak')]
[('charactername', 'Jim Halpert'), ('actorname', 'John Krasinski')]
[('charactername', 'Pam Halpert'), ('actorname', 'Jenna Fischer')]
[('charactername', 'Michael Scott'), ('actorname', 'Steve Carell')]
[('charactername', 'Dwight Schrute'), ('actorname', 'Rainn Wilson')]
[('charactername', 'Andy Bernard'), ('actorname', 'Ed Helms')]
[('charactername', 'Jan Levinson'), ('actorname', 'Melora Hardin')]
[('charactername', 'Robert California'), ('actorname', 'James Spader')]
[('charactername', 'Erin Hannon'), ('actorname', 'Ellie Kemper')]
[('charactername', 'Kelly Kapoor'), ('actorname', 'Mindy Kaling')]
[('charactername', 'Kevin Malone'), ('actorname', 'Brian Baumgartner')]
[('charactername', 'Meredith Palmer'), ('actorname', 'Henriette Mantel')]
[('charactername', 'Roy Anderson'), ('actorname', 'David Denman')]
[('charactername', 'Michael Scarn'), ('actorname', 'Steve Carell')]
[('charactername', 'Mose Schrute'), ('

30

In [29]:

# Show the name of all the characters of both series, their actors,
# count the number of the characters 

queryString = """
SELECT (COUNT (DISTINCT ?character) AS ?countcharacter) 
WHERE { 

    ?character wdt:P1441 wd:Q23831 .
    ?character wdt:P175 ?actor .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?character <http://schema.org/name> ?charactername .
    
} 

"""

print("Results")
run_query(queryString)

Results
[('countcharacter', '33')]


1

In [30]:
# Show the name of all the characters of both series, their actors
# and the other occupution every actor has.

queryString = """
SELECT ?charactername ?actorname ?occupationname
WHERE { 

    ?character wdt:P1441 wd:Q23831 .
    ?character wdt:P175 ?actor .
    ?actor wdt:P106 ?occupation .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?character <http://schema.org/name> ?charactername .
    ?occupation <http://schema.org/name> ?occupationname .

} 
ORDER BY ?charactername
LIMIT 30
"""

print("Results")
run_query(queryString)



Results
[('charactername', 'Andy Bernard'), ('actorname', 'Ed Helms'), ('occupationname', 'banjoist')]
[('charactername', 'Andy Bernard'), ('actorname', 'Ed Helms'), ('occupationname', 'television actor')]
[('charactername', 'Andy Bernard'), ('actorname', 'Ed Helms'), ('occupationname', 'film actor')]
[('charactername', 'Andy Bernard'), ('actorname', 'Ed Helms'), ('occupationname', 'voice actor')]
[('charactername', 'Andy Bernard'), ('actorname', 'Ed Helms'), ('occupationname', 'comedian')]
[('charactername', 'Andy Bernard'), ('actorname', 'Ed Helms'), ('occupationname', 'screenwriter')]
[('charactername', 'Andy Bernard'), ('actorname', 'Ed Helms'), ('occupationname', 'character actor')]
[('charactername', 'Angela Martin'), ('actorname', 'Angela Kinsey'), ('occupationname', 'television actor')]
[('charactername', 'Angela Martin'), ('actorname', 'Angela Kinsey'), ('occupationname', 'film actor')]
[('charactername', 'Angela Martin'), ('actorname', 'Angela Kinsey'), ('occupationname', 'po

30

In [31]:
# Show the name of all the characters of both series, their actors
# and the other occupution every actor has.

queryString = """
SELECT ?charactername ?actorname ?occupationname
WHERE { 

    ?character wdt:P1441 wd:Q23831 .
    ?character wdt:P175 ?actor .
    ?actor wdt:P106 ?occupation .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?character <http://schema.org/name> ?charactername .
    ?occupation <http://schema.org/name> ?occupationname .
    FILTER regex(?occupationname, "singer", "i") 

} 
ORDER BY ?charactername
LIMIT 30
"""

print("Results")
run_query(queryString)



Results
[('charactername', 'Creed Bratton'), ('actorname', 'Creed Bratton'), ('occupationname', 'singer-songwriter')]
[('charactername', 'Darryl Philbin'), ('actorname', 'Craig Robinson'), ('occupationname', 'singer')]
[('charactername', 'Jan Levinson'), ('actorname', 'Melora Hardin'), ('occupationname', 'singer')]
[('charactername', 'President Jackson'), ('actorname', 'Craig Robinson'), ('occupationname', 'singer')]
[('charactername', 'Todd Packer'), ('actorname', 'David Koechner'), ('occupationname', 'singer')]


5

# Task 3

## Find the movies of the actors of each series.

In [32]:
# actor name and finding its occupation
#[('actor', 'http://www.wikidata.org/entity/Q200566'),
#('actorname', 'Cobie Smulders')]
queryString = """
SELECT *
WHERE { 

     ?p ?o wd:Q200566 .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
} 
LIMIT 10
"""

print("Results")
run_query(queryString)
# find the movies of the actor and count them 

Results
[('p', 'http://www.wikidata.org/entity/Q65070140'), ('o', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'Stumptown'), ('oname', 'cast member')]
[('p', 'http://www.wikidata.org/entity/Q51963292'), ('o', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'Marvel Cinematic Universe Phase One'), ('oname', 'cast member')]
[('p', 'http://www.wikidata.org/entity/Q63405798'), ('o', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'The Infinity Saga'), ('oname', 'cast member')]
[('p', 'http://www.wikidata.org/entity/Q147235'), ('o', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'How I Met Your Mother'), ('oname', 'cast member')]
[('p', 'http://www.wikidata.org/entity/Q5264968'), ('o', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'Desperation Day'), ('oname', 'cast member')]
[('p', 'http://www.wikidata.org/entity/Q5521981'), ('o', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'Garbage Island'), ('oname', 'cast member')]
[('p', 'http://www.wi

10

In [33]:
# actor name and finding its occupation
#[('actor', 'http://www.wikidata.org/entity/Q200566'),
#('actorname', 'Cobie Smulders')]
queryString = """
SELECT *
WHERE { 

     ?p ?o wd:Q200566 .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?oname, "cast", "i") 
} 
LIMIT 10
"""

print("Results")
run_query(queryString)
# find the movies of the actor and count them 

Results
[('p', 'http://www.wikidata.org/entity/Q65070140'), ('o', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'Stumptown'), ('oname', 'cast member')]
[('p', 'http://www.wikidata.org/entity/Q51963292'), ('o', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'Marvel Cinematic Universe Phase One'), ('oname', 'cast member')]
[('p', 'http://www.wikidata.org/entity/Q63405798'), ('o', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'The Infinity Saga'), ('oname', 'cast member')]
[('p', 'http://www.wikidata.org/entity/Q147235'), ('o', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'How I Met Your Mother'), ('oname', 'cast member')]
[('p', 'http://www.wikidata.org/entity/Q5264968'), ('o', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'Desperation Day'), ('oname', 'cast member')]
[('p', 'http://www.wikidata.org/entity/Q5521981'), ('o', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'Garbage Island'), ('oname', 'cast member')]
[('p', 'http://www.wi

10

In [34]:
# find the movies of the actor and count them 
# actor name and finding its occupation
#[('actor', 'http://www.wikidata.org/entity/Q200566'),
#('actorname', 'Cobie Smulders')]
queryString = """
SELECT *
WHERE { 

    ?movie wdt:P161 wd:Q200566 .
    # Labels
    ?movie <http://schema.org/name> ?moviename .
} 
LIMIT 10
"""

print("Results")
run_query(queryString)


Results
[('movie', 'http://www.wikidata.org/entity/Q65070140'), ('moviename', 'Stumptown')]
[('movie', 'http://www.wikidata.org/entity/Q51963292'), ('moviename', 'Marvel Cinematic Universe Phase One')]
[('movie', 'http://www.wikidata.org/entity/Q63405798'), ('moviename', 'The Infinity Saga')]
[('movie', 'http://www.wikidata.org/entity/Q147235'), ('moviename', 'How I Met Your Mother')]
[('movie', 'http://www.wikidata.org/entity/Q5264968'), ('moviename', 'Desperation Day')]
[('movie', 'http://www.wikidata.org/entity/Q5521981'), ('moviename', 'Garbage Island')]
[('movie', 'http://www.wikidata.org/entity/Q7080458'), ('moviename', 'Oh Honey')]
[('movie', 'http://www.wikidata.org/entity/Q5888848'), ('moviename', 'Home Wreckers')]
[('movie', 'http://www.wikidata.org/entity/Q7858076'), ('moviename', 'Twin Beds')]
[('movie', 'http://www.wikidata.org/entity/Q8074081'), ('moviename', 'Zoo or False')]


10

In [35]:
# marvel
# actor name and finding its occupation
#[('actor', 'http://www.wikidata.org/entity/Q200566'),
#('actorname', 'Cobie Smulders')]
queryString = """
SELECT *
WHERE { 

    ?movie wdt:P161 wd:Q200566 .
    # Labels
    ?movie <http://schema.org/name> ?moviename .
    FILTER regex(?moviename, "marvel", "i") 
    
} 
LIMIT 10
"""

print("Results")
run_query(queryString)
# marvel

Results
[('movie', 'http://www.wikidata.org/entity/Q51963292'), ('moviename', 'Marvel Cinematic Universe Phase One')]


1

### HIMYM

In [36]:
# HIMYM
queryString = """
SELECT *
WHERE { 

    ?character wdt:P1441 wd:Q147235 .
    ?character wdt:P175 ?actor .
    ?movie wdt:P161 ?actor .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?character <http://schema.org/name> ?charactername .
    
    ?movie <http://schema.org/name> ?moviename .

} 
limit 10
"""

print("Results")
run_query(queryString)


Results
[('character', 'http://www.wikidata.org/entity/Q50383228'), ('actor', 'http://www.wikidata.org/entity/Q4888924'), ('movie', 'http://www.wikidata.org/entity/Q22908690'), ('actorname', 'Benjamin Koldyke'), ('charactername', 'Don Frank'), ('moviename', 'The Good Place')]
[('character', 'http://www.wikidata.org/entity/Q50383228'), ('actor', 'http://www.wikidata.org/entity/Q4888924'), ('movie', 'http://www.wikidata.org/entity/Q147235'), ('actorname', 'Benjamin Koldyke'), ('charactername', 'Don Frank'), ('moviename', 'How I Met Your Mother')]
[('character', 'http://www.wikidata.org/entity/Q50383228'), ('actor', 'http://www.wikidata.org/entity/Q4888924'), ('movie', 'http://www.wikidata.org/entity/Q18031184'), ('actorname', 'Benjamin Koldyke'), ('charactername', 'Don Frank'), ('moviename', 'The Finest Hours')]
[('character', 'http://www.wikidata.org/entity/Q50383228'), ('actor', 'http://www.wikidata.org/entity/Q4888924'), ('movie', 'http://www.wikidata.org/entity/Q128730'), ('actorname

10

In [37]:
# HIMYM
queryString = """
SELECT ?actorname ?countmovie
WHERE { 

      {
              SELECT ?actor (COUNT (DISTINCT ?movie) AS ?countmovie) WHERE
              {
                    ?character wdt:P1441 wd:Q147235 .
                    ?character wdt:P175 ?actor .
                    ?movie wdt:P161 ?actor .

               }
                GROUP BY ?actor
      }  .

    # Labels
    ?actor <http://schema.org/name> ?actorname .

} 
ORDER BY DESC(?countmovie) ?actorname
LIMIT 50
"""

print("Results")
run_query(queryString)


Results
[('actorname', 'Neil Patrick Harris'), ('countmovie', '187')]
[('actorname', 'Cobie Smulders'), ('countmovie', '173')]
[('actorname', 'Alyson Hannigan'), ('countmovie', '170')]
[('actorname', 'Jason Segel'), ('countmovie', '170')]
[('actorname', 'Josh Radnor'), ('countmovie', '155')]
[('actorname', 'John Lithgow'), ('countmovie', '74')]
[('actorname', 'David Henrie'), ('countmovie', '67')]
[('actorname', 'Seth Green'), ('countmovie', '66')]
[('actorname', 'Ray Wise'), ('countmovie', '64')]
[('actorname', 'Lyndsy Fonseca'), ('countmovie', '63')]
[('actorname', 'Peter Gallagher'), ('countmovie', '62')]
[('actorname', 'Jane Seymour'), ('countmovie', '60')]
[('actorname', 'Bob Odenkirk'), ('countmovie', '54')]
[('actorname', 'Kyle MacLachlan'), ('countmovie', '54')]
[('actorname', 'Jennifer Morrison'), ('countmovie', '53')]
[('actorname', 'Bryan Cranston'), ('countmovie', '52')]
[('actorname', 'Frances Conroy'), ('countmovie', '49')]
[('actorname', 'John Cho'), ('countmovie', '48')

50

### The office

In [38]:
# The office
queryString = """
SELECT *
WHERE { 
    ?character wdt:P1441 wd:Q23831 .
    ?character wdt:P175 ?actor .
    ?movie wdt:P161 ?actor .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?character <http://schema.org/name> ?charactername .
    
    ?movie <http://schema.org/name> ?moviename .
} 
limit 10
"""

print("Results")
run_query(queryString)


Results
[('character', 'http://www.wikidata.org/entity/Q7372521'), ('actor', 'http://www.wikidata.org/entity/Q64560'), ('movie', 'http://www.wikidata.org/entity/Q17480384'), ('actorname', 'David Denman'), ('charactername', 'Roy Anderson'), ('moviename', 'Take')]
[('character', 'http://www.wikidata.org/entity/Q2013030'), ('actor', 'http://www.wikidata.org/entity/Q313039'), ('movie', 'http://www.wikidata.org/entity/Q18150704'), ('actorname', 'John Krasinski'), ('charactername', 'Jim Halpert'), ('moviename', 'The Hollars')]
[('character', 'http://www.wikidata.org/entity/Q63801326'), ('actor', 'http://www.wikidata.org/entity/Q313039'), ('movie', 'http://www.wikidata.org/entity/Q18150704'), ('actorname', 'John Krasinski'), ('charactername', 'Goldenface'), ('moviename', 'The Hollars')]
[('character', 'http://www.wikidata.org/entity/Q4762500'), ('actor', 'http://www.wikidata.org/entity/Q2669971'), ('movie', 'http://www.wikidata.org/entity/Q22908828'), ('actorname', 'Angela Kinsey'), ('charact

10

In [39]:
queryString = """
SELECT ?actorname ?countmovie
WHERE { 

      {
              SELECT ?actor (COUNT (DISTINCT ?movie) AS ?countmovie) WHERE
              {
                    ?character wdt:P1441 wd:Q23831 .
                    ?character wdt:P175 ?actor .
                    ?movie wdt:P161 ?actor .

               }
                GROUP BY ?actor
      }  .

    # Labels
    ?actor <http://schema.org/name> ?actorname .

} 
ORDER BY DESC(?countmovie) ?actorname
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('actorname', 'David Koechner'), ('countmovie', '63')]
[('actorname', 'Will Ferrell'), ('countmovie', '60')]
[('actorname', 'James Spader'), ('countmovie', '46')]
[('actorname', 'Steve Carell'), ('countmovie', '44')]
[('actorname', 'Catherine Tate'), ('countmovie', '37')]
[('actorname', 'Melora Hardin'), ('countmovie', '37')]
[('actorname', 'John Krasinski'), ('countmovie', '35')]
[('actorname', 'Amy Ryan'), ('countmovie', '34')]
[('actorname', 'Craig Robinson'), ('countmovie', '33')]
[('actorname', 'Ed Helms'), ('countmovie', '33')]
[('actorname', 'David Denman'), ('countmovie', '31')]
[('actorname', 'Rainn Wilson'), ('countmovie', '29')]
[('actorname', 'Jenna Fischer'), ('countmovie', '26')]
[('actorname', 'Clark Duke'), ('countmovie', '21')]
[('actorname', 'Jake Lacy'), ('countmovie', '16')]
[('actorname', 'Mindy Kaling'), ('countmovie', '15')]
[('actorname', 'Ellie Kemper'), ('countmovie', '14')]
[('actorname', 'Oscar Nunez'), ('countmovie', '12')]
[('actorname', 'Zach Woo

28

# Task 4

# Show the name of the spouses of the actors in every series and show the occupution of their spouses. and filter singers.

In [40]:
# HIMYM
queryString = """
SELECT *
WHERE { 


    wd:Q200566 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .

} 
limit 20
"""

print("Results")
run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('o', 'http://www.wikidata.org/entity/Q15208593'), ('pname', 'given name'), ('oname', 'Francisca')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('o', 'http://www.wikidata.org/entity/Q6119607'), ('pname', 'given name'), ('oname', 'Jacoba')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('o', 'http://www.wikidata.org/entity/Q10798782'), ('pname', 'occupation'), ('oname', 'television actor')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('o', 'http://www.wikidata.org/entity/Q10800557'), ('pname', 'occupation'), ('oname', 'film actor')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('o', 'http://www.wikidata.org/entity/Q16'), ('pname', 'country of citizenship'), ('oname', 'Canada')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('pname', 'languages spoken, written or signed'), ('oname', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('

15

In [41]:
# HIMYM
queryString = """
SELECT *
WHERE { 


    wd:Q200566 wdt:P26 ?spouse .
    # Labels
    ?spouse <http://schema.org/name> ?spousename .

} 
limit 20
"""

print("Results")
run_query(queryString)


Results
[('spouse', 'http://www.wikidata.org/entity/Q2706805'), ('spousename', 'Taran Killam')]


1

In [42]:
# HIMYM
queryString = """
SELECT *
WHERE { 


    wd:Q2706805 ?o ?p .
    # Labels
    ?o <http://schema.org/name> ?oname .
    ?p <http://schema.org/name> ?pname .

} 
limit 20
"""

print("Results")
run_query(queryString)


Results
[('o', 'http://www.wikidata.org/prop/direct/P26'), ('p', 'http://www.wikidata.org/entity/Q200566'), ('oname', 'spouse'), ('pname', 'Cobie Smulders')]
[('o', 'http://www.wikidata.org/prop/direct/P69'), ('p', 'http://www.wikidata.org/entity/Q6682090'), ('oname', 'educated at'), ('pname', 'Los Angeles County High School for the Arts')]
[('o', 'http://www.wikidata.org/prop/direct/P106'), ('p', 'http://www.wikidata.org/entity/Q10798782'), ('oname', 'occupation'), ('pname', 'television actor')]
[('o', 'http://www.wikidata.org/prop/direct/P106'), ('p', 'http://www.wikidata.org/entity/Q10800557'), ('oname', 'occupation'), ('pname', 'film actor')]
[('o', 'http://www.wikidata.org/prop/direct/P69'), ('p', 'http://www.wikidata.org/entity/Q174710'), ('oname', 'educated at'), ('pname', 'University of California, Los Angeles')]
[('o', 'http://www.wikidata.org/prop/direct/P106'), ('p', 'http://www.wikidata.org/entity/Q2405480'), ('oname', 'occupation'), ('pname', 'voice actor')]
[('o', 'http:/

16

In [43]:
# HIMYM
queryString = """
SELECT *
WHERE { 


    wd:Q2706805 wdt:P106 ?p .
    # Labels
    ?p <http://schema.org/name> ?pname .

} 
LIMIT 20
"""

print("Results")
run_query(queryString)


Results
[('p', 'http://www.wikidata.org/entity/Q10798782'), ('pname', 'television actor')]
[('p', 'http://www.wikidata.org/entity/Q10800557'), ('pname', 'film actor')]
[('p', 'http://www.wikidata.org/entity/Q2405480'), ('pname', 'voice actor')]
[('p', 'http://www.wikidata.org/entity/Q245068'), ('pname', 'comedian')]
[('p', 'http://www.wikidata.org/entity/Q2526255'), ('pname', 'film director')]
[('p', 'http://www.wikidata.org/entity/Q36180'), ('pname', 'writer')]
[('p', 'http://www.wikidata.org/entity/Q578109'), ('pname', 'television producer')]


7

## HIMYM

In [44]:
# HIMYM
queryString = """
SELECT DISTINCT ?actorname ?spousename ?spouseoccname
WHERE { 

    ?character wdt:P1441 wd:Q147235 .
    ?character wdt:P175 ?actor .
    ?movie wdt:P161 ?actor .
    ?actor wdt:P26 ?spouse .
    ?spouse wdt:P106 ?spouseocc .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?character <http://schema.org/name> ?charactername .
    ?movie <http://schema.org/name> ?moviename .
    ?spouse <http://schema.org/name> ?spousename .
    ?spouseocc <http://schema.org/name> ?spouseoccname
    FILTER regex(?spouseoccname, "singer", "i") 
} 
LIMIT 10
"""

print("Results")
run_query(queryString)


### order the name of the actor by the most episodes they have acted in this series

Results
[('actorname', 'Britney Spears'), ('spousename', 'Kevin Federline'), ('spouseoccname', 'singer')]
[('actorname', 'Jennifer Lopez'), ('spousename', 'Marc Anthony'), ('spouseoccname', 'singer')]
[('actorname', 'Bill Fagerbakke'), ('spousename', 'Catherine McClenahan'), ('spouseoccname', 'singer')]
[('actorname', 'Katy Perry'), ('spousename', 'Russell Brand'), ('spouseoccname', 'singer')]
[('actorname', 'David Burtka'), ('spousename', 'Neil Patrick Harris'), ('spouseoccname', 'singer')]
[('actorname', 'Seth Green'), ('spousename', 'Clare Grant'), ('spouseoccname', 'singer')]
[('actorname', 'Martin Short'), ('spousename', 'Nancy Dolman'), ('spouseoccname', 'singer')]


7

## The office

In [45]:
# HIMYM
queryString = """
SELECT ?actorname ?spousename ?spouseoccname
WHERE { 

    ?character wdt:P1441 wd:Q23831 .
    ?character wdt:P175 ?actor .
    ?movie wdt:P161 ?actor .
    ?actor wdt:P26 ?spouse .
    ?spouse wdt:P106 ?spouseocc .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?character <http://schema.org/name> ?charactername .
    ?movie <http://schema.org/name> ?moviename .
    ?spouse <http://schema.org/name> ?spousename .
    ?spouseocc <http://schema.org/name> ?spouseoccname

} 
LIMIT 10
"""

print("Results")
run_query(queryString)


### order the name of the actor by the most episodes they have acted in this series

Results
[('actorname', 'Steve Carell'), ('spousename', 'Nancy Carell'), ('spouseoccname', 'television actor')]
[('actorname', 'Steve Carell'), ('spousename', 'Nancy Carell'), ('spouseoccname', 'television actor')]
[('actorname', 'Steve Carell'), ('spousename', 'Nancy Carell'), ('spouseoccname', 'film actor')]
[('actorname', 'Steve Carell'), ('spousename', 'Nancy Carell'), ('spouseoccname', 'film actor')]
[('actorname', 'Steve Carell'), ('spousename', 'Nancy Carell'), ('spouseoccname', 'screenwriter')]
[('actorname', 'Steve Carell'), ('spousename', 'Nancy Carell'), ('spouseoccname', 'screenwriter')]
[('actorname', 'Steve Carell'), ('spousename', 'Nancy Carell'), ('spouseoccname', 'actor')]
[('actorname', 'Steve Carell'), ('spousename', 'Nancy Carell'), ('spouseoccname', 'actor')]
[('actorname', 'Steve Carell'), ('spousename', 'Nancy Carell'), ('spouseoccname', 'television producer')]
[('actorname', 'Steve Carell'), ('spousename', 'Nancy Carell'), ('spouseoccname', 'television producer')

10

## Get the number of episodes in which the cast members played a role. Who are the most present actors?

### HIMYM

In [46]:
# show cast members
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q147235 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?pname, "cast member", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q465556'), ('oname', 'Annie Ilonzeh')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q269891'), ('oname', 'Julianna Guill')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q4888924'), ('oname', 'Benjamin Koldyke')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q446031'), ('oname', 'Nikki Griffin')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q312705'), ('oname', 'John Cho')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q469579'), ('oname', 'Mircea Monroe')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 

20

In [47]:
### instance of the cast members:
# ('o', 'http://www.wikidata.org/entity/Q465556'), ('oname', 'Annie Ilonzeh')]

queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q465556 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth'), ('o', 'http://www.wikidata.org/entity/Q185216'), ('oname', 'Grapevine')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10798782'), ('oname', 'television actor')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10800557'), ('oname', 'film actor')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pname', 'country of citizenship'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oname', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q33999'), ('oname', 'actor')]
[('p', 'http://www.wikidata.org/prop/direct/P172'), ('pname', 'ethnic group'), ('o', 'http://www.wikidata.org/entity/Q49085'), ('oname', 'African Americans')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), 

12

In [48]:
### instance of the cast members:
# ('o', 'http://www.wikidata.org/entity/Q465556'), ('oname', 'Annie Ilonzeh')]

queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    ?p ?o wd:Q465556 .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?pname, "how I met your mother", "i") 
    
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q147235'), ('pname', 'How I Met Your Mother'), ('o', 'http://www.wikidata.org/prop/direct/P161'), ('oname', 'cast member')]


1

In [49]:
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q147235 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?pname, "episode", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1811'), ('pname', 'list of episodes'), ('o', 'http://www.wikidata.org/entity/Q785891'), ('oname', 'list of How I Met Your Mother episodes')]


1

In [50]:
#Count episodes of a season of HIMYM and show the number of seasons

queryString = """
    SELECT ?countepisode ?seasonname WHERE {
      {
        SELECT ?season (COUNT (DISTINCT ?episode) AS ?countepisode) 
        WHERE { 
            wd:Q147235 wdt:P527 ?season .
            ?season wdt:P527 ?episode .

                } 
    GROUP BY ?season
    } .
     ## labels   
    ?season <http://schema.org/name> ?seasonname .
    }
    ORDER BY (?countepisode) ?seasonname 


"""

print("Results")
run_query(queryString)

Results
[('countepisode', '20'), ('seasonname', 'How I Met Your Mother, season 3')]
[('countepisode', '22'), ('seasonname', 'How I Met Your Mother, season 1')]
[('countepisode', '22'), ('seasonname', 'How I Met Your Mother, season 2')]
[('countepisode', '24'), ('seasonname', 'How I Met Your Mother, season 9')]
[('countepisode', '24'), ('seasonname', 'How I Met Your Mother, season 7')]
[('countepisode', '24'), ('seasonname', 'How I Met Your Mother, season 6')]
[('countepisode', '24'), ('seasonname', 'How I Met Your Mother, season 8')]
[('countepisode', '24'), ('seasonname', 'How I Met Your Mother, season 4')]
[('countepisode', '24'), ('seasonname', 'How I Met Your Mother, season 5')]


9

In [51]:
#Count episodes of a season of HIMYM and show the number of seasons

queryString = """

SELECT ?season ?seasonname ?episode ?episodename
WHERE { 
            wd:Q147235 wdt:P527 ?season .
            ?season wdt:P527 ?episode .
            ?season <http://schema.org/name> ?seasonname .
            ?episode <http://schema.org/name> ?episodename .
        } 
GROUP BY ?season
LIMIT 10 



"""

print("Results")
run_query(queryString)

Results
[('season', 'http://www.wikidata.org/entity/Q2715578'), ('seasonname', 'How I Met Your Mother, season 1'), ('episode', 'http://www.wikidata.org/entity/Q467447'), ('episodename', 'Pilot')]
[('season', 'http://www.wikidata.org/entity/Q582332'), ('seasonname', 'How I Met Your Mother, season 5'), ('episode', 'http://www.wikidata.org/entity/Q7538739'), ('episodename', 'Slapsgiving 2: Revenge of the Slap')]
[('season', 'http://www.wikidata.org/entity/Q2555117'), ('seasonname', 'How I Met Your Mother, season 3'), ('episode', 'http://www.wikidata.org/entity/Q7776702'), ('episodename', 'The Yips')]
[('season', 'http://www.wikidata.org/entity/Q13567027'), ('seasonname', 'How I Met Your Mother, season 9'), ('episode', 'http://www.wikidata.org/entity/Q16746127'), ('episodename', 'The Locket')]
[('season', 'http://www.wikidata.org/entity/Q13567027'), ('seasonname', 'How I Met Your Mother, season 9'), ('episode', 'http://www.wikidata.org/entity/Q16746145'), ('episodename', 'Coming Back')]
[(

10

In [52]:
#[('season', 'http://www.wikidata.org/entity/Q2715578'),
# ('seasonname', 'How I Met Your Mother, season 1'),
# ('episode', 'http://www.wikidata.org/entity/Q467447'),
# ('episodename', 'Pilot')]
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q467447 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?pname, "cast member", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q200566'), ('oname', 'Cobie Smulders')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q485310'), ('oname', 'Neil Patrick Harris')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q223455'), ('oname', 'Josh Radnor')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q199927'), ('oname', 'Alyson Hannigan')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q202304'), ('oname', 'Jason Segel')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q552889'), ('oname', 'Marshall Manesh')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('p

9

In [53]:
#[('p', 'http://www.wikidata.org/prop/direct/P161'), 
#('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q485310'), ('oname', 'Neil Patrick Harris')]
queryString = """
SELECT ?p ?pname 
WHERE { 
    
    ?p wdt:P161 wd:Q485310  .
    # Labels
    ?p <http://schema.org/name> ?pname .

    #FILTER regex(?pname, "cast member", "i") 
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q147235'), ('pname', 'How I Met Your Mother')]
[('p', 'http://www.wikidata.org/entity/Q5264968'), ('pname', 'Desperation Day')]
[('p', 'http://www.wikidata.org/entity/Q5521981'), ('pname', 'Garbage Island')]
[('p', 'http://www.wikidata.org/entity/Q7080458'), ('pname', 'Oh Honey')]
[('p', 'http://www.wikidata.org/entity/Q5888848'), ('pname', 'Home Wreckers')]
[('p', 'http://www.wikidata.org/entity/Q7858076'), ('pname', 'Twin Beds')]
[('p', 'http://www.wikidata.org/entity/Q8074081'), ('pname', 'Zoo or False')]
[('p', 'http://www.wikidata.org/entity/Q1253366'), ('pname', 'The Smurfs 2')]
[('p', 'http://www.wikidata.org/entity/Q842253'), ('pname', "Dr. Horrible's Sing-Along Blog")]
[('p', 'http://www.wikidata.org/entity/Q2093198'), ('pname', 'Undercover Brother')]


10

In [54]:
#Count episodes of a season of HIMYM and show the number of seasons

queryString = """
    SELECT?countepisode ?seasonname WHERE {
      {
        SELECT ?season (COUNT (DISTINCT ?episode) AS ?countepisode) 
        WHERE { 
            wd:Q147235 wdt:P527 ?season .
            ?season wdt:P527 ?episode .

                } 
    GROUP BY ?season
    } .
     ## labels   
    ?season <http://schema.org/name> ?seasonname .
    }
    ORDER BY (?countepisode) ?seasonname 


"""

print("Results")
run_query(queryString)

Results
[('countepisode', '20'), ('seasonname', 'How I Met Your Mother, season 3')]
[('countepisode', '22'), ('seasonname', 'How I Met Your Mother, season 1')]
[('countepisode', '22'), ('seasonname', 'How I Met Your Mother, season 2')]
[('countepisode', '24'), ('seasonname', 'How I Met Your Mother, season 9')]
[('countepisode', '24'), ('seasonname', 'How I Met Your Mother, season 7')]
[('countepisode', '24'), ('seasonname', 'How I Met Your Mother, season 6')]
[('countepisode', '24'), ('seasonname', 'How I Met Your Mother, season 8')]
[('countepisode', '24'), ('seasonname', 'How I Met Your Mother, season 4')]
[('countepisode', '24'), ('seasonname', 'How I Met Your Mother, season 5')]


9

In [55]:
#Count episodes of a season of HIMYM and show the number of seasons

queryString = """

        SELECT ?seasonname ?episode ?episodename
        WHERE { 
            wd:Q147235 wdt:P527 ?season .
            ?season wdt:P527 ?episode .
            ?season <http://schema.org/name> ?seasonname .
            ?episode <http://schema.org/name> ?episodename .

                } 
                LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('seasonname', 'How I Met Your Mother, season 6'), ('episode', 'http://www.wikidata.org/entity/Q5264968'), ('episodename', 'Desperation Day')]
[('seasonname', 'How I Met Your Mother, season 6'), ('episode', 'http://www.wikidata.org/entity/Q5521981'), ('episodename', 'Garbage Island')]
[('seasonname', 'How I Met Your Mother, season 6'), ('episode', 'http://www.wikidata.org/entity/Q7080458'), ('episodename', 'Oh Honey')]
[('seasonname', 'How I Met Your Mother, season 5'), ('episode', 'http://www.wikidata.org/entity/Q5888848'), ('episodename', 'Home Wreckers')]
[('seasonname', 'How I Met Your Mother, season 5'), ('episode', 'http://www.wikidata.org/entity/Q7858076'), ('episodename', 'Twin Beds')]
[('seasonname', 'How I Met Your Mother, season 5'), ('episode', 'http://www.wikidata.org/entity/Q8074081'), ('episodename', 'Zoo or False')]
[('seasonname', 'How I Met Your Mother, season 6'), ('episode', 'http://www.wikidata.org/entity/Q4786988'), ('episodename', 'Architect of Destructi

10

In [56]:
#Q8074081' episode name
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q8074081 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .



    #FILTER regex(?pname, "episode", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q200566'), ('oname', 'Cobie Smulders')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q439070'), ('oname', 'E. E. Bell')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('pname', 'part of the series'), ('o', 'http://www.wikidata.org/entity/Q147235'), ('oname', 'How I Met Your Mother')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('pname', 'director'), ('o', 'http://www.wikidata.org/entity/Q460221'), ('oname', 'Pamela Fryman')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pname', 'followed by'), ('o', 'http://www.wikidata.org/entity/Q5888848'), ('oname', 'Home Wreckers')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('pname', 'original language of film or TV show'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('oname', 'English')]
[('p', 'http://www.wikidata.org/pro

18

In [57]:
#Q8074081' episode name
queryString = """
SELECT ?episodename ?castname
WHERE { 
    wd:Q147235 wdt:P527 ?season .
    ?season wdt:P527 ?episode .
    ?episode wdt:P161 ?cast .
    # Labels
    #?season <http://schema.org/name> ?seasonname .
    ?episode <http://schema.org/name> ?episodename .
    ?cast <http://schema.org/name> ?castname .



    #FILTER regex(?pname, "episode", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('episodename', 'Desperation Day'), ('castname', 'Cobie Smulders')]
[('episodename', 'Desperation Day'), ('castname', 'Bill Fagerbakke')]
[('episodename', 'Desperation Day'), ('castname', 'Nazanin Boniadi')]
[('episodename', 'Desperation Day'), ('castname', 'Jennifer Morrison')]
[('episodename', 'Desperation Day'), ('castname', 'Jesse Heiman')]
[('episodename', 'Desperation Day'), ('castname', 'Neil Patrick Harris')]
[('episodename', 'Desperation Day'), ('castname', 'Josh Radnor')]
[('episodename', 'Desperation Day'), ('castname', 'Suzie Plakson')]
[('episodename', 'Desperation Day'), ('castname', 'Alyson Hannigan')]
[('episodename', 'Desperation Day'), ('castname', 'Jason Segel')]
[('episodename', 'Desperation Day'), ('castname', 'Bob Saget')]
[('episodename', 'Garbage Island'), ('castname', 'Cobie Smulders')]
[('episodename', 'Garbage Island'), ('castname', 'Nazanin Boniadi')]
[('episodename', 'Garbage Island'), ('castname', 'Jennifer Morrison')]
[('episodename', 'Garbage Is

20

In [58]:
#Q8074081' episode name
queryString = """
SELECT ?episodename ?castname ?cast
WHERE { 
    wd:Q147235 wdt:P527 ?season .
    ?season wdt:P527 ?episode .
    ?episode wdt:P161 ?cast .
    # Labels
    #?season <http://schema.org/name> ?seasonname .
    ?episode <http://schema.org/name> ?episodename .
    ?cast <http://schema.org/name> ?castname .



    #FILTER regex(?pname, "episode", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('episodename', 'Desperation Day'), ('castname', 'Cobie Smulders'), ('cast', 'http://www.wikidata.org/entity/Q200566')]
[('episodename', 'Desperation Day'), ('castname', 'Bill Fagerbakke'), ('cast', 'http://www.wikidata.org/entity/Q591238')]
[('episodename', 'Desperation Day'), ('castname', 'Nazanin Boniadi'), ('cast', 'http://www.wikidata.org/entity/Q1189102')]
[('episodename', 'Desperation Day'), ('castname', 'Jennifer Morrison'), ('cast', 'http://www.wikidata.org/entity/Q199929')]
[('episodename', 'Desperation Day'), ('castname', 'Jesse Heiman'), ('cast', 'http://www.wikidata.org/entity/Q978367')]
[('episodename', 'Desperation Day'), ('castname', 'Neil Patrick Harris'), ('cast', 'http://www.wikidata.org/entity/Q485310')]
[('episodename', 'Desperation Day'), ('castname', 'Josh Radnor'), ('cast', 'http://www.wikidata.org/entity/Q223455')]
[('episodename', 'Desperation Day'), ('castname', 'Suzie Plakson'), ('cast', 'http://www.wikidata.org/entity/Q1396070')]
[('episodename', '

20

In [59]:
# 'Nazanin Boniadi' ('cast', 'http://www.wikidata.org/entity/Q1189102')

queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q1189102 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .

} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10798782'), ('oname', 'television actor')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10800557'), ('oname', 'film actor')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pname', 'country of citizenship'), ('o', 'http://www.wikidata.org/entity/Q145'), ('oname', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q33999'), ('oname', 'actor')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth'), ('o', 'http://www.wikidata.org/entity/Q3616'), ('oname', 'Tehran')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('pname', 'religion'), ('o', 'http://www.wikidata.org/entity/Q432'), ('oname', 'Islam')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 

14

In [60]:
# the number of episodes a cast played # Q1189102 'Nazanin Boniadi' 

queryString = """
SELECT (COUNT (DISTINCT ?episode) AS ?countepisode) WHERE
{ 
    wd:Q147235 wdt:P527 ?season .
    ?season wdt:P527 ?episode .
    ?episode wdt:P161 wd:Q1189102 .
    # Labels
    #?season <http://schema.org/name> ?seasonname .
    ?episode <http://schema.org/name> ?episodename .
    #?cast <http://schema.org/name> ?castname .
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('countepisode', '8')]


1

## Get the number of HIMYM episodes in which the cast members played a role. Who are the most present actors?

In [61]:
# the number of episodes a cast played 

queryString = """
SELECT ?castname ?cast (COUNT (DISTINCT ?episode) AS ?countepisode) WHERE
{ 
    wd:Q147235 wdt:P527 ?season .
    ?season wdt:P527 ?episode .
    ?episode wdt:P161 ?cast .
    # Labels
    ?season <http://schema.org/name> ?seasonname .
    ?episode <http://schema.org/name> ?episodename .
    ?cast <http://schema.org/name> ?castname .
} 
ORDER BY DESC (?countepisode)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('castname', 'Cobie Smulders'), ('cast', 'http://www.wikidata.org/entity/Q200566'), ('countepisode', '145')]
[('castname', 'Neil Patrick Harris'), ('cast', 'http://www.wikidata.org/entity/Q485310'), ('countepisode', '145')]
[('castname', 'Josh Radnor'), ('cast', 'http://www.wikidata.org/entity/Q223455'), ('countepisode', '145')]
[('castname', 'Jason Segel'), ('cast', 'http://www.wikidata.org/entity/Q202304'), ('countepisode', '145')]
[('castname', 'Alyson Hannigan'), ('cast', 'http://www.wikidata.org/entity/Q199927'), ('countepisode', '143')]
[('castname', 'Bob Saget'), ('cast', 'http://www.wikidata.org/entity/Q333544'), ('countepisode', '142')]
[('castname', 'David Henrie'), ('cast', 'http://www.wikidata.org/entity/Q297128'), ('countepisode', '48')]
[('castname', 'Lyndsy Fonseca'), ('cast', 'http://www.wikidata.org/entity/Q229914'), ('countepisode', '48')]
[('castname', 'Charlene Amoia'), ('cast', 'http://www.wikidata.org/entity/Q16149506'), ('countepisode', '17')]
[('castnam

20

## Get the number of The office episodes in which the cast members played a role. Who are the most present actors?

In [62]:
# the number of episodes a cast played 

queryString = """
SELECT ?castname (COUNT (DISTINCT ?episode) AS ?countepisode) WHERE
{ 
    wd:Q23831 wdt:P527 ?season .
    ?season wdt:P527 ?episode .
    ?episode wdt:P161 ?cast .
    # Labels
    ?season <http://schema.org/name> ?seasonname .
    ?episode <http://schema.org/name> ?episodename .
    ?cast <http://schema.org/name> ?castname .
} 
ORDER BY DESC (?countepisode)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('castname', 'Ellie Kemper'), ('countepisode', '1')]
[('castname', 'Craig Robinson'), ('countepisode', '1')]
[('castname', 'Catherine Tate'), ('countepisode', '1')]
[('castname', 'Jenna Fischer'), ('countepisode', '1')]
[('castname', 'Rainn Wilson'), ('countepisode', '1')]
[('castname', 'Ed Helms'), ('countepisode', '1')]
[('castname', 'John Krasinski'), ('countepisode', '1')]


7

In [63]:
# the number of episodes a cast played 

queryString = """
SELECT ?episode ?episodename WHERE
{ 
    wd:Q23831 wdt:P527 ?season .
    ?season wdt:P527 ?episode .
    # Labels
    ?season <http://schema.org/name> ?seasonname .
    ?episode <http://schema.org/name> ?episodename .
    #?cast <http://schema.org/name> ?castname .
} 
LIMIT 20

"""

print("Results")
run_query(queryString)

Results
[('episode', 'http://www.wikidata.org/entity/Q50379836'), ('episodename', 'Classy Christmas (part 1)')]
[('episode', 'http://www.wikidata.org/entity/Q50379837'), ('episodename', 'Classy Christmas (part 2)')]
[('episode', 'http://www.wikidata.org/entity/Q5099551'), ('episodename', 'China')]
[('episode', 'http://www.wikidata.org/entity/Q5178024'), ('episodename', 'Couples Discount')]
[('episode', 'http://www.wikidata.org/entity/Q6927074'), ('episodename', 'Moving On')]
[('episode', 'http://www.wikidata.org/entity/Q7880294'), ('episodename', 'Ultimatum')]
[('episode', 'http://www.wikidata.org/entity/Q7914333'), ('episodename', 'Vandalism')]
[('episode', 'http://www.wikidata.org/entity/Q4838397'), ('episodename', 'Baby Shower')]
[('episode', 'http://www.wikidata.org/entity/Q5001718'), ('episodename', 'Business Ethics')]
[('episode', 'http://www.wikidata.org/entity/Q5185169'), ('episodename', 'Crime Aid')]
[('episode', 'http://www.wikidata.org/entity/Q4956350'), ('episodename', 'Bra

20

In [64]:
#Count episodes of a season of The Office and show the number of seasons

queryString = """
    SELECT ?seasonname ?countepisode WHERE {
      {
        SELECT ?season (COUNT (DISTINCT ?episode) AS ?countepisode) 
        WHERE { 
            wd:Q23831 wdt:P527 ?season .
            ?season wdt:P527 ?episode .

                } 
    GROUP BY ?season
    } .
     ## labels   
    ?season <http://schema.org/name> ?seasonname .
    }
    ORDER BY (?seasonname) ?countepisode


"""

print("Results")
run_query(queryString)

Results
[('seasonname', 'The Office, season 1'), ('countepisode', '6')]
[('seasonname', 'The Office, season 2'), ('countepisode', '22')]
[('seasonname', 'The Office, season 3'), ('countepisode', '25')]
[('seasonname', 'The Office, season 4'), ('countepisode', '19')]
[('seasonname', 'The Office, season 5'), ('countepisode', '28')]
[('seasonname', 'The Office, season 6'), ('countepisode', '26')]
[('seasonname', 'The Office, season 7'), ('countepisode', '26')]
[('seasonname', 'The Office, season 8'), ('countepisode', '24')]
[('seasonname', 'The Office, season 9'), ('countepisode', '25')]


9

In [65]:
# Q50379836
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q23831 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?pname, "cast", "i") 
} 
LIMIT 50
"""

print("Results")
run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P449'), ('pname', 'original broadcaster'), ('o', 'http://www.wikidata.org/entity/Q13974'), ('oname', 'NBC')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q1050211'), ('oname', 'Leslie David Baker')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q1139248'), ('oname', 'Oscar Nunez')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q216221'), ('oname', 'Steve Carell')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q2238008'), ('oname', 'Creed Bratton')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q231203'), ('oname', 'Amy Ryan')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'c

26

In [66]:
# Q50379836
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    ?p ?o wd:Q1050211 .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .

} 
LIMIT 20
"""

print("Results")
run_query(queryString)


Results
[('p', 'http://www.wikidata.org/entity/Q2529789'), ('pname', 'Screen Actors Guild Award for Outstanding Performance by an Ensemble in a Comedy Series'), ('o', 'http://www.wikidata.org/prop/direct/P1346'), ('oname', 'winner')]
[('p', 'http://www.wikidata.org/entity/Q23831'), ('pname', 'The Office'), ('o', 'http://www.wikidata.org/prop/direct/P161'), ('oname', 'cast member')]
[('p', 'http://www.wikidata.org/entity/Q7599690'), ('pname', 'Stanley Hudson'), ('o', 'http://www.wikidata.org/prop/direct/P175'), ('oname', 'performer')]


3

In [67]:
# Q50379836
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q1050211 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .

} 
LIMIT 20
"""

print("Results")
run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received'), ('o', 'http://www.wikidata.org/entity/Q2529789'), ('oname', 'Screen Actors Guild Award for Outstanding Performance by an Ensemble in a Comedy Series')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q2529789'), ('oname', 'Screen Actors Guild Award for Outstanding Performance by an Ensemble in a Comedy Series')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('pname', 'given name'), ('o', 'http://www.wikidata.org/entity/Q26736517'), ('oname', 'Leslie')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10798782'), ('oname', 'television actor')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10800557'), ('oname', 'film actor')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'pla

14

In [68]:
# Q50379836
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q50379836 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .

} 
LIMIT 20
"""

print("Results")
run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pname', 'followed by'), ('o', 'http://www.wikidata.org/entity/Q50379837'), ('oname', 'Classy Christmas (part 2)')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows'), ('o', 'http://www.wikidata.org/entity/Q5099551'), ('oname', 'China')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of'), ('o', 'http://www.wikidata.org/entity/Q5128465'), ('oname', 'Classy Christmas')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('pname', 'original language of film or TV show'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('oname', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q21191270'), ('oname', 'television series episode')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pname', 'country of origin'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oname', 'United States of America')]
[('p', 'http://w

10

In [69]:
# Q50379836
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

     ?p ?o wd:Q50379836 .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .

} 
LIMIT 20
"""

print("Results")
run_query(queryString)


Results
[('p', 'http://www.wikidata.org/entity/Q50379837'), ('pname', 'Classy Christmas (part 2)'), ('o', 'http://www.wikidata.org/prop/direct/P155'), ('oname', 'follows')]
[('p', 'http://www.wikidata.org/entity/Q5128465'), ('pname', 'Classy Christmas'), ('o', 'http://www.wikidata.org/prop/direct/P527'), ('oname', 'has part')]
[('p', 'http://www.wikidata.org/entity/Q3730263'), ('pname', 'The Office, season 7'), ('o', 'http://www.wikidata.org/prop/direct/P527'), ('oname', 'has part')]


3

## Check who is the actor who acted in more films while working on "How I met your mother" and who is the actor who participated in more films after the end of the tv series.

In [70]:
# HIMYM
queryString = """
SELECT ?actorname ?countmovie
WHERE { 

      {
              SELECT ?actor (COUNT (DISTINCT ?movie) AS ?countmovie) WHERE
              {
                    ?character wdt:P1441 wd:Q147235 .
                    ?character wdt:P175 ?actor .
                    ?movie wdt:P161 ?actor .

               }
                GROUP BY ?actor
      }  .

    # Labels
    ?actor <http://schema.org/name> ?actorname .

} 
ORDER BY DESC(?countmovie) ?actorname
LIMIT 20
"""

print("Results")
run_query(queryString)


Results
[('actorname', 'Neil Patrick Harris'), ('countmovie', '187')]
[('actorname', 'Cobie Smulders'), ('countmovie', '173')]
[('actorname', 'Alyson Hannigan'), ('countmovie', '170')]
[('actorname', 'Jason Segel'), ('countmovie', '170')]
[('actorname', 'Josh Radnor'), ('countmovie', '155')]
[('actorname', 'John Lithgow'), ('countmovie', '74')]
[('actorname', 'David Henrie'), ('countmovie', '67')]
[('actorname', 'Seth Green'), ('countmovie', '66')]
[('actorname', 'Ray Wise'), ('countmovie', '64')]
[('actorname', 'Lyndsy Fonseca'), ('countmovie', '63')]
[('actorname', 'Peter Gallagher'), ('countmovie', '62')]
[('actorname', 'Jane Seymour'), ('countmovie', '60')]
[('actorname', 'Bob Odenkirk'), ('countmovie', '54')]
[('actorname', 'Kyle MacLachlan'), ('countmovie', '54')]
[('actorname', 'Jennifer Morrison'), ('countmovie', '53')]
[('actorname', 'Bryan Cranston'), ('countmovie', '52')]
[('actorname', 'Frances Conroy'), ('countmovie', '49')]
[('actorname', 'John Cho'), ('countmovie', '48')

20

In [71]:
## finding the start time and end time of every series. 

In [72]:
#wd:Q147235
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q147235 ?p ?o .

    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?pname, "time", "i")

} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [73]:
# | `wd:Q23831` | The Office (US)        | node |
queryString = """
SELECT COUNT(DISTINCT ?p)
WHERE { 

    wd:Q23831 ?p ?o .

    # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .

} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '77')]


1

In [74]:
# | `wd:Q23831` | The Office (US)        | node |
queryString = """
SELECT ?p ?pname
WHERE { 

    wd:Q23831 ?p ?o .

    # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    FILTER regex(?pname, "time", "i")

} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('pname', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('pname', 'end time')]


2

In [75]:
#P580 start time
#P582 end time
# | `wd:Q23831` | The Office (US)        | node |
queryString = """
SELECT *
WHERE { 

    wd:Q23831 wdt:P580 ?start_time .
    wd:Q23831 wdt:P582 ?end_time .
    # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .

} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('start_time', '2005-03-24T00:00:00Z'), ('end_time', '2013-05-16T00:00:00Z')]


1

In [76]:

queryString = """
SELECT ?actorname ?countmovie
WHERE { 

      {
              SELECT ?actor (COUNT (DISTINCT ?movie) AS ?countmovie) WHERE
              {
                    ?character wdt:P1441 wd:Q147235 .
                    ?character wdt:P175 ?actor .
                    ?movie wdt:P161 ?actor .

               }
                GROUP BY ?actor
      }  .

    # Labels
    ?actor <http://schema.org/name> ?actorname .

} 
ORDER BY DESC(?countmovie) ?actorname
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('actorname', 'Neil Patrick Harris'), ('countmovie', '187')]
[('actorname', 'Cobie Smulders'), ('countmovie', '173')]
[('actorname', 'Alyson Hannigan'), ('countmovie', '170')]
[('actorname', 'Jason Segel'), ('countmovie', '170')]
[('actorname', 'Josh Radnor'), ('countmovie', '155')]
[('actorname', 'John Lithgow'), ('countmovie', '74')]
[('actorname', 'David Henrie'), ('countmovie', '67')]
[('actorname', 'Seth Green'), ('countmovie', '66')]
[('actorname', 'Ray Wise'), ('countmovie', '64')]
[('actorname', 'Lyndsy Fonseca'), ('countmovie', '63')]


10

In [77]:
###    wd:Q23831 wdt:P580 ?start_time .
###    wd:Q23831 wdt:P582 ?end_time .

queryString = """
SELECT ?movie ?moviename ?start_time ?end_time
WHERE { 

    ?character wdt:P1441 wd:Q147235 .
    ?character wdt:P175 ?actor .
    ?movie wdt:P161 ?actor .
    ?movie wdt:P580 ?start_time ;
            wdt:P582 ?end_time .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?movie <http://schema.org/name> ?moviename .
    #FILTER regex(?pname, "ww", "i") 
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('movie', 'http://www.wikidata.org/entity/Q22908690'), ('moviename', 'The Good Place'), ('start_time', '2016-09-19T00:00:00Z'), ('end_time', '2020-01-30T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q147235'), ('moviename', 'How I Met Your Mother'), ('start_time', '2005-09-19T00:00:00Z'), ('end_time', '2014-03-31T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q23558'), ('moviename', 'House'), ('start_time', '2004-11-16T00:00:00Z'), ('end_time', '2012-05-21T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q147235'), ('moviename', 'How I Met Your Mother'), ('start_time', '2005-09-19T00:00:00Z'), ('end_time', '2014-03-31T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q844059'), ('moviename', 'New Girl'), ('start_time', '2011-09-20T00:00:00Z'), ('end_time', '2018-05-15T00:00:00Z')]
[('movie', 'http://www.wikidata.org/entity/Q476857'), ('moviename', 'FlashForward'), ('start_time', '2009-09-24T00:00:00Z'), ('end_time', '2010-05-27T00:00:00Z')]
[('m

10

### Check who is the actor who acted in more films while working on "How I met your mother"

In [78]:
queryString = """
SELECT ?actorname ?moviename ?start_time ?end_time
WHERE { 

    ?character wdt:P1441 wd:Q147235 .
    ?character wdt:P175 ?actor .
    ?movie wdt:P161 ?actor .
    ?movie wdt:P580 ?start_time ;
            wdt:P582 ?end_time .
    wd:Q147235 wdt:P580 ?himym_start .
    wd:Q147235 wdt:P582 ?himym_end .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?movie <http://schema.org/name> ?moviename .
    #FILTER regex(?pname, "ww", "i") 
    FILTER (?himym_start < ?start_time)
    FILTER (?end_time < ?himym_end)
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('actorname', 'John Cho'), ('moviename', 'FlashForward'), ('start_time', '2009-09-24T00:00:00Z'), ('end_time', '2010-05-27T00:00:00Z')]
[('actorname', 'Mircea Monroe'), ('moviename', 'Rules of Engagement'), ('start_time', '2007-02-05T00:00:00Z'), ('end_time', '2013-05-20T00:00:00Z')]
[('actorname', 'Mircea Monroe'), ('moviename', 'Chuck'), ('start_time', '2007-09-24T00:00:00Z'), ('end_time', '2012-01-27T00:00:00Z')]
[('actorname', 'Mircea Monroe'), ('moviename', 'Zeke and Luther'), ('start_time', '2009-06-15T00:00:00Z'), ('end_time', '2012-04-02T00:00:00Z')]
[('actorname', 'Mircea Monroe'), ('moviename', 'Drive'), ('start_time', '2007-04-13T00:00:00Z'), ('end_time', '2007-04-23T00:00:00Z')]
[('actorname', 'Virginia Williams'), ('moviename', 'Rules of Engagement'), ('start_time', '2007-02-05T00:00:00Z'), ('end_time', '2013-05-20T00:00:00Z')]
[('actorname', 'Virginia Williams'), ('moviename', 'Better Off Ted'), ('start_time', '2009-03-18T00:00:00Z'), ('end_time', '2010-01-26T00:

10

In [79]:
queryString = """
SELECT ?actorname (COUNT(?movie) AS ?countmovies)
WHERE { 

    ?character wdt:P1441 wd:Q147235 .
    ?character wdt:P175 ?actor .
    ?movie wdt:P161 ?actor .
    ?movie wdt:P580 ?start_time ;
            wdt:P582 ?end_time .
    wd:Q147235 wdt:P580 ?himym_start .
    wd:Q147235 wdt:P582 ?himym_end .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?movie <http://schema.org/name> ?moviename .
    ?character <http://schema.org/name> ?charactername .
    FILTER (?himym_start < ?start_time)
    FILTER (?end_time < ?himym_end)
} 

ORDER BY DESC (?countmovies) 

LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('actorname', 'Amy Acker'), ('countmovies', '6')]
[('actorname', 'Virginia Williams'), ('countmovies', '4')]
[('actorname', 'Mircea Monroe'), ('countmovies', '4')]
[('actorname', 'Abigail Spencer'), ('countmovies', '4')]
[('actorname', 'Lucy Hale'), ('countmovies', '4')]
[('actorname', 'Frances Conroy'), ('countmovies', '4')]
[('actorname', 'Darcy Rose Byrnes'), ('countmovies', '4')]
[('actorname', 'Cristine Rose'), ('countmovies', '3')]
[('actorname', 'Bryan Callen'), ('countmovies', '3')]
[('actorname', 'Geoff Stults'), ('countmovies', '3')]


10

### Who is the actor who participated in more films after the end of the HIMYM.

In [80]:
queryString = """
SELECT ?actorname (COUNT(?movie) AS ?countmovies)
WHERE { 

    ?character wdt:P1441 wd:Q147235 .
    ?character wdt:P175 ?actor .
    ?movie wdt:P161 ?actor .
    ?movie wdt:P580 ?start_time ;
            wdt:P582 ?end_time .
    wd:Q147235 wdt:P580 ?himym_start .
    wd:Q147235 wdt:P582 ?himym_end .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?movie <http://schema.org/name> ?moviename .
    ?character <http://schema.org/name> ?charactername .
    FILTER (?himym_end < ?start_time)
} 

ORDER BY DESC (?countmovies) 

LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('actorname', 'Josh Radnor'), ('countmovies', '6')]
[('actorname', 'Neil Patrick Harris'), ('countmovies', '6')]
[('actorname', 'Kal Penn'), ('countmovies', '4')]
[('actorname', 'Amy Acker'), ('countmovies', '3')]
[('actorname', 'Taran Killam'), ('countmovies', '3')]
[('actorname', 'Virginia Williams'), ('countmovies', '2')]
[('actorname', 'Kyle MacLachlan'), ('countmovies', '2')]
[('actorname', 'Cobie Smulders'), ('countmovies', '2')]
[('actorname', 'Ray Wise'), ('countmovies', '2')]
[('actorname', 'JoAnna García'), ('countmovies', '2')]


10

### Check who is the actor who acted in more films while working on "The office".

In [81]:
queryString = """
SELECT ?actorname (COUNT(?movie) AS ?countmovies)
WHERE { 

    ?character wdt:P1441 wd:Q23831 .
    ?character wdt:P175 ?actor .
    ?movie wdt:P161 ?actor .
    ?movie wdt:P580 ?start_time ;
            wdt:P582 ?end_time .
    wd:Q23831 wdt:P580 ?theoffice_start .
    wd:Q23831 wdt:P582 ?theoffice_end .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?movie <http://schema.org/name> ?moviename .
    ?character <http://schema.org/name> ?charactername .
    FILTER (?theoffice_start < ?start_time)
    FILTER (?end_time < ?theoffice_end)
} 

ORDER BY DESC (?countmovies) 

LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('actorname', 'David Koechner'), ('countmovies', '2')]
[('actorname', 'Oscar Nunez'), ('countmovies', '1')]
[('actorname', 'Melora Hardin'), ('countmovies', '1')]
[('actorname', 'Amy Ryan'), ('countmovies', '1')]
[('actorname', 'Clark Duke'), ('countmovies', '1')]


5

### Who is the actor who participated in more films after the end of the The office.

In [82]:
queryString = """
SELECT ?actorname (COUNT(?movie) AS ?countmovies)
WHERE { 

    ?character wdt:P1441 wd:Q147235 .
    ?character wdt:P175 ?actor .
    ?movie wdt:P161 ?actor .
    ?movie wdt:P580 ?start_time ;
            wdt:P582 ?end_time .
    wd:Q147235 wdt:P580 ?theoffice_start .
    wd:Q147235 wdt:P582 ?theoffice_end .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    #?movie <http://schema.org/name> ?moviename .
    #?character <http://schema.org/name> ?charactername .
    FILTER (?theoffice_end < ?start_time)
} 

ORDER BY DESC (?countmovies) 

LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('actorname', 'Josh Radnor'), ('countmovies', '6')]
[('actorname', 'Neil Patrick Harris'), ('countmovies', '6')]
[('actorname', 'Kal Penn'), ('countmovies', '4')]
[('actorname', 'Amy Acker'), ('countmovies', '3')]
[('actorname', 'Taran Killam'), ('countmovies', '3')]
[('actorname', 'Virginia Williams'), ('countmovies', '2')]
[('actorname', 'Kyle MacLachlan'), ('countmovies', '2')]
[('actorname', 'Cobie Smulders'), ('countmovies', '2')]
[('actorname', 'Ray Wise'), ('countmovies', '2')]
[('actorname', 'JoAnna García'), ('countmovies', '2')]


10

In [83]:
queryString = """
SELECT ?actor (COUNT(?movie) AS ?countmovies)
WHERE { 

    ?character wdt:P1441 wd:Q147235 .
    ?character wdt:P175 ?actor .
    ?movie wdt:P161 ?actor .
    ?movie wdt:P580 ?start_time ;
            wdt:P582 ?end_time .
    wd:Q147235 wdt:P580 ?theoffice_start .
    wd:Q147235 wdt:P582 ?theoffice_end .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?movie <http://schema.org/name> ?moviename .
    ?character <http://schema.org/name> ?charactername .
    FILTER (?theoffice_end < ?start_time)
} 

ORDER BY DESC (?countmovies) 

LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q223455'), ('countmovies', '6')]
[('actor', 'http://www.wikidata.org/entity/Q485310'), ('countmovies', '6')]
[('actor', 'http://www.wikidata.org/entity/Q220536'), ('countmovies', '4')]
[('actor', 'http://www.wikidata.org/entity/Q2706805'), ('countmovies', '3')]
[('actor', 'http://www.wikidata.org/entity/Q152773'), ('countmovies', '3')]
[('actor', 'http://www.wikidata.org/entity/Q207506'), ('countmovies', '2')]
[('actor', 'http://www.wikidata.org/entity/Q200566'), ('countmovies', '2')]
[('actor', 'http://www.wikidata.org/entity/Q131332'), ('countmovies', '2')]
[('actor', 'http://www.wikidata.org/entity/Q530646'), ('countmovies', '2')]
[('actor', 'http://www.wikidata.org/entity/Q516659'), ('countmovies', '2')]


10

# Task 5

### Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2.

In [84]:
#?movie wdt:P161 ?actor .

queryString = """
SELECT DISTINCT ?movie ?actorname ?actor
WHERE { 
    
    ?movie wdt:P161 ?actor .
    wd:Q147235 ?p ?o .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    FILTER regex(?actorname, "kevin bacon", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('movie', 'http://www.wikidata.org/entity/Q122113'), ('actorname', 'Kevin Bacon'), ('actor', 'http://www.wikidata.org/entity/Q3454165')]
[('movie', 'http://www.wikidata.org/entity/Q463927'), ('actorname', 'Kevin Bacon'), ('actor', 'http://www.wikidata.org/entity/Q3454165')]
[('movie', 'http://www.wikidata.org/entity/Q636062'), ('actorname', 'Kevin Bacon'), ('actor', 'http://www.wikidata.org/entity/Q3454165')]
[('movie', 'http://www.wikidata.org/entity/Q691672'), ('actorname', 'Kevin Bacon'), ('actor', 'http://www.wikidata.org/entity/Q3454165')]
[('movie', 'http://www.wikidata.org/entity/Q1917925'), ('actorname', 'Kevin Bacon'), ('actor', 'http://www.wikidata.org/entity/Q3454165')]
[('movie', 'http://www.wikidata.org/entity/Q5265117'), ('actorname', 'Kevin Bacon'), ('actor', 'http://www.wikidata.org/entity/Q3454165')]
[('movie', 'http://www.wikidata.org/entity/Q1154802'), ('actorname', 'Kevin Bacon'), ('actor', 'http://www.wikidata.org/entity/Q3454165')]
[('movie', 'http://www.

20

In [85]:
#?movie wdt:P161 ?actor .
# Q3454165 KEVIN BACON
queryString = """
SELECT COUNT (DISTINCT ?movie)
WHERE { 
    
    ?movie wdt:P161 ?actor .
    wd:Q147235 ?p ?o .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    FILTER regex(?actorname, "kevin bacon", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '69')]


1

In [86]:
#?movie wdt:P161 ?actor .
# Q3454165 KEVIN BACON
queryString = """
SELECT COUNT (DISTINCT ?movie)
WHERE { 
    
    ?moviekb wdt:P161 wd:Q3454165 .
    wd:Q147235 ?p ?o .
    # Labels
    #?actor <http://schema.org/name> ?actorname .
    FILTER regex(?actorname, "kevin bacon", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '0')]


1

In [87]:
## Kevin bacon = 1 'HIMYM'

In [88]:
queryString = """
SELECT ?actorname ?moviename
WHERE { 

    ?character wdt:P1441 wd:Q147235 .
    ?character wdt:P175 ?actor .
    ?movie wdt:P161 ?actor .
    ?moviekb wdt:P161 wd:Q3454165 .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?movie <http://schema.org/name> ?moviename .
    #FILTER regex(?pname, "ww", "i") 
    FILTER (?moviekb = ?movie)
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('actorname', 'John Cho'), ('moviename', 'The Air I Breathe')]
[('actorname', 'John Lithgow'), ('moviename', 'Footloose')]
[('actorname', 'Ray Wise'), ('moviename', 'X-Men: First Class')]
[('actorname', 'Jennifer Morrison'), ('moviename', 'Stir of Echoes')]
[('actorname', 'Jennifer Morrison'), ('moviename', 'The Darkness')]
[('actorname', 'Martin Short'), ('moviename', 'The Big Picture')]
[('actorname', 'Michael Gross'), ('moviename', 'Tremors')]
[('actorname', 'Sherri Shepherd'), ('moviename', 'Beauty Shop')]


8

In [89]:
# Kevin bacon number = 2
queryString = """
SELECT DISTINCT ?actorname
WHERE { 

    ?character wdt:P1441 wd:Q147235 .
    ?character wdt:P175 ?actor .
    ?movie wdt:P161 ?actor .
    ?moviekb wdt:P161 wd:Q3454165 .
    ?moviekb wdt:P161 ?actorkb .
    ?moviekbtwo wdt:P161 ?actorkb .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?movie <http://schema.org/name> ?moviename .
    FILTER (?moviekbtwo = ?movie)
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('actorname', 'John Cho')]
[('actorname', 'Eva Amurri')]
[('actorname', 'Jayma Mays')]
[('actorname', 'Darcy Rose Byrnes')]
[('actorname', 'Laura Prepon')]
[('actorname', 'Benjamin Koldyke')]
[('actorname', 'Nazanin Boniadi')]
[('actorname', 'Katie Holmes')]
[('actorname', 'Geoff Stults')]
[('actorname', 'Jennifer Morrison')]


10

### HIMYM actors kevin bacon number 2.
### how many of the actors who are members of the cast of the tv series have [Kavin Bacon number] equal to 2

In [90]:
# Kevin bacon number = 2
queryString = """
SELECT (COUNT (DISTINCT ?actorname) AS ?countkbnumber)
WHERE { 

    ?character wdt:P1441 wd:Q147235 .
    ?character wdt:P175 ?actor .
    ?movie wdt:P161 ?actor .
    ?moviekb wdt:P161 wd:Q3454165 .
    ?moviekb wdt:P161 ?actorkb .
    ?moviekbtwo wdt:P161 ?actorkb .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?movie <http://schema.org/name> ?moviename .
    FILTER (?moviekbtwo = ?movie)
} 

"""

print("Results")
run_query(queryString)

Results
[('countkbnumber', '110')]


1

### The office actors kevin bacon number 2.

In [91]:
# Kevin bacon number = 2
queryString = """
SELECT DISTINCT ?actorname
WHERE { 

    ?character wdt:P1441 wd:Q23831 .
    ?character wdt:P175 ?actor .
    ?movie wdt:P161 ?actor .
    ?moviekb wdt:P161 wd:Q3454165 .
    ?moviekb wdt:P161 ?actorkb .
    ?moviekbtwo wdt:P161 ?actorkb .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?movie <http://schema.org/name> ?moviename .
    FILTER (?moviekbtwo = ?movie)
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('actorname', 'David Koechner')]
[('actorname', 'Jenna Fischer')]
[('actorname', 'Phyllis Smith')]
[('actorname', 'Craig Robinson')]
[('actorname', 'Will Ferrell')]
[('actorname', 'Melora Hardin')]
[('actorname', 'James Spader')]
[('actorname', 'David Denman')]
[('actorname', 'Oscar Nunez')]
[('actorname', 'Ed Helms')]


10

In [92]:
# Kevin bacon number = 2
queryString = """
SELECT (COUNT (DISTINCT ?actorname) AS ?countkbnumber)
WHERE { 

    ?character wdt:P1441 wd:Q23831 .
    ?character wdt:P175 ?actor .
    ?movie wdt:P161 ?actor .
    ?moviekb wdt:P161 wd:Q3454165 .
    ?moviekb wdt:P161 ?actorkb .
    ?moviekbtwo wdt:P161 ?actorkb .
    # Labels
    ?actor <http://schema.org/name> ?actorname .
    ?movie <http://schema.org/name> ?moviename .
    FILTER (?moviekbtwo = ?movie)
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('countkbnumber', '28')]


1